## HW4 Image Filtering: Difference of Gaussians

The Laplacian of Gaussian (LoG) kernel can be approximated by a scaled Difference of Gaussians (DoG) kernel, cf. [B&B Book 3, Sec 7.1.1]. That is,

\begin{equation}
L_\sigma(x,y) \approx \frac{2k^2}{\sigma^2 (k^2-1)} ~( G_{k\sigma}(x,y) - G_{\sigma}(x,y) ),
\qquad
G_{\sigma} = \frac{1}{2\pi\sigma^2} e^{-(x^2+y^2)/(2\sigma^2)}
\end{equation}

ECE472: Implement an LoG kernel using skimage.filters.gaussian() and skimage.filters.laplace(). Apply to F35.gif for sigma=1.0. Repeat for the DoG kernel skimage.filters.difference_of_gaussians(). Determine determine sigma values for the Dog kernel that approximate the appearance of LoG filtering.

ECE572: Complete the ECE472 assignment only implement your own DoG kernel using the above formulae.

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io
from skimage import filters as flt
from skimage.util import img_as_float32 as img_as_float

from scipy.ndimage import convolve

In [ ]:
def print_imginfo(I):
    print(type(I))
    print(I.shape, I.dtype)
    print('Data range:', np.min(I), 'to', np.max(I))

In [ ]:
def show_imghist(I, vmin=0.0, vmax=1.0):
    fig, ax = plt.subplots(1, 2, figsize=(10,3))
    
    ax[0].imshow(I, cmap='gray', vmin=vmin, vmax=vmax)
    ax[0].set_axis_off()
    
    ax[1].hist(I.ravel(), lw=0, bins=256, range=(vmin,vmax));
    ax[1].set_xlim(vmin, vmax)
    ax[1].set_yticks([])

In [ ]:
I1 = io.imread("../images/F35.gif", as_gray=True)
I1 = img_as_float(I1)

print_imginfo(I1)
show_imghist(I1)

## ECE472 Students: SKImage DoG Kernel

In [ ]:
truncate = 4.0
mode = 'reflect'

sigma = 1.0
k = 2

I2 = flt.gaussian(I1, sigma, truncate=truncate, mode=mode)
I2 = flt.laplace(I2)

I3 = flt.difference_of_gaussians(I1, sigma, k*sigma, truncate=truncate, mode=mode)

print_imginfo(I2)
print('')
print_imginfo(I3)

show_imghist(I2, vmin=-0.03, vmax=0.03)
show_imghist(I3, vmin=-0.03, vmax=0.03)

## ECE572 Students:  Homebrew DoG Kernel

In [ ]:
I2 = flt.gaussian(I1, sigma, truncate=truncate, mode=mode)
I2 = flt.laplace(I2)

def DoG_kernel(sigma=1.0, k=1.6, truncate=3.0):
    s1 = sigma
    s2 = k*sigma
    
    N = np.int32(np.round(truncate*s2))
    u, v = np.mgrid[-N:N+1,-N:N+1]
    
    h1 = 1.0/(2*np.pi*s1**2)*np.exp(-0.5*(u**2+v**2)/(s1**2))
    h2 = 1.0/(2*np.pi*s2**2)*np.exp(-0.5*(u**2+v**2)/(s2**2))
    
    h = (2*k**2)/(np.pi*(sigma**2)*(k**2-1))*(h2-h1)

    return h

h = DoG_kernel(sigma=sigma, k=k, truncate=truncate)
I3 = convolve(I1, h, mode=mode)

print_imginfo(I2)
print('')
print_imginfo(I3)

show_imghist(I2, vmin=-0.03, vmax=0.03)
show_imghist(I3, vmin=-0.03, vmax=0.03)